In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
df = pd.read_csv('output.csv')

In [3]:
df

,Unnamed: 0,Name,Suit,Points
0,0,Queen of Hearts,Hearts,0.03
1,1,9 of Diamonds,Diamonds,2.00
2,2,10 of Spades,Spades,1.00
3,3,Queen of Clubs,Clubs,0.03
4,4,7 of Clubs,Clubs,0.01
5,5,Ace of Hearts,Hearts,1.10
6,6,10 of Diamonds,Diamonds,1.00
7,7,9 of Clubs,Clubs,2.00
8,8,10 of Clubs,Clubs,1.00
9,9,8 of Hearts,Hearts,0.02


In [4]:
df = df.drop('Unnamed: 0', axis = 1)

In [5]:
class Player():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False

class Robot():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False
class Game():
    def __init__(self):
        self.color = None
        self.last_hand_winner = None
        self.digged = False


In [6]:
class Bid():
    def __init__(self, b1, b2):
        self.how_much_b1 = 17
        self.how_much_b2 = 17
        self.b1_bid = input(f"Do you want to bid {b1.name}? Y/N: ").lower()
        
        if self.b1_bid == "n":
            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
            self.winner = b2
        elif self.b1_bid == "y":
            self.b2_bid = input(f"Do you want to bid {b2.name}? Y/N: ").lower()

            if self.b2_bid == "n":
                print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                self.winner = b1
            else:
                while self.b1_bid == "y" and self.b2_bid == "y":
                    self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                    self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Ensure minimum bid
                    while self.how_much_b1 < 17 or self.how_much_b2 < 17:
                        print("Invalid bid! The minimum bid is 17.")
                        self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                        self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Determine winner of the bid
                    if self.how_much_b1 < self.how_much_b2:
                        self.b1_bid = input(f"{b2.name} is winning the bid. Do you want to continue, {b1.name}? Y/N: ").lower()
                        if self.b1_bid == "n":
                            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
                            self.winner = b2
                    elif self.how_much_b1 > self.how_much_b2:
                        self.b2_bid = input(f"{b1.name} is winning the bid. Do you want to continue, {b2.name}? Y/N: ").lower()
                        if self.b2_bid == "n":
                            print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                            self.winner = b1


In [7]:
game = Game()

In [8]:
def shuffle_deck():
    global df
    df = df.sample(frac = 1, ignore_index = True)

In [9]:
def distribute_cards():
    global df
    global player, robot_1, robot_2, robot_3
    player = Player(df[0:8], input("What is your name? "))
    robot_1 = Robot(df[8:16], 'Robot 1')
    robot_2 = Robot(df[16:24], 'Robot 2')
    robot_3 = Robot(df[24:32], 'Robot 3')

In [10]:
def play_card(name, player):
    global current_hand
    current_hand = pd.concat([current_hand, df[df['Name'] == name]], ignore_index = True)
    # pd.concat([new_data, data], ignore_index=True)
    player.cards = player.cards.drop(player.cards[player.cards['Name'] == name].index)

In [35]:
def setup():
    
    shuffle_deck() # Shuffling all the cards
    distribute_cards() # Distributing all the cards
    print("Nice to meet you {}".format(player.name))
    print("Welcome to our newly made game of 29. We hope you enjoy it")
    print(player.cards[:4]) # Showing the player their first 4 cards
    global game
    game = Game()
        
    # BIDDING
    player_bid = int(input('How much do you want to bid?: '))
    
    # robot_1
    r1_bid = 0
    if len(robot_1.cards[:4]['Suit'].mode() == 1):
        r1_mode_suit = robot_1.cards[:4]['Suit'].mode().iloc[0]
        r1_color = robot_1.cards[:4]['Suit'].value_counts()[r1_mode_suit]
        r1_points = robot_1.cards[:4][robot_1.cards[:4]['Suit'] == r1_mode_suit]['Points'].sum()
        
        if r1_points == 3 or r1_points == 3.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 4 or r1_points == 4.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 5 or r1_points == 5.1:
            if r1_color == 2: r1_bid = 19
            if r1_color == 3: r1_bid = 20
        if r1_points == 6 or r1_points == 6.1:
            if r1_color == 3: r1_bid = 21
        if r1_points == 6 or r1_points == 6.1 or r1_points == 7.1:
            if r1_color == 4: r1_bid = 22
    
    # robot_2
    r2_bid = 0
    if len(robot_2.cards[:4]['Suit'].mode() == 2):
        r2_mode_suit = robot_2.cards[:4]['Suit'].mode().iloc[0]
        r2_color = robot_2.cards[:4]['Suit'].value_counts()[r2_mode_suit]
        r2_points = robot_2.cards[:4][robot_2.cards[:4]['Suit'] == r2_mode_suit]['Points'].sum()
        
        if r2_points == 3 or r2_points == 3.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 4 or r2_points == 4.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 5 or r2_points == 5.1:
            if r2_color == 2: r2_bid = 19
            if r2_color == 3: r2_bid = 20
        if r2_points == 6 or r2_points == 6.1:
            if r2_color == 3: r2_bid = 21
        if r2_points == 6 or r2_points == 6.1 or r2_points == 7.1:
            if r2_color == 4: r2_bid = 22
    
    # robot_3
    r3_bid = 0
    if len(robot_3.cards[:4]['Suit'].mode() == 3):
        r3_mode_suit = robot_3.cards[:4]['Suit'].mode().iloc[0]
        r3_color = robot_3.cards[:4]['Suit'].value_counts()[r3_mode_suit]
        r3_points = robot_3.cards[:4][robot_3.cards[:4]['Suit'] == r3_mode_suit]['Points'].sum()
        
        if r3_points == 3 or r3_points == 3.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 4 or r3_points == 4.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 5 or r3_points == 5.1:
            if r3_color == 2: r3_bid = 19
            if r3_color == 3: r3_bid = 20
        if r3_points == 6 or r3_points == 6.1:
            if r3_color == 3: r3_bid = 22
        if r3_points == 6 or r3_points == 6.1 or r3_points == 7.1:
            if r3_color == 4: r3_bid = 22
    bidding = [player_bid, r1_bid, r2_bid, r3_bid]
    max_bid = max(bidding)
    max_bid_index = bidding.index(max_bid)
    if max_bid <17:
        r3_bid = 17
        winner = robot_3.name
        max_bid = r3_bid
    else:
        if max_bid_index == 0: 
            winner = player.name
        if max_bid_index == 1: 
            winner = robot_1.name
        if max_bid_index == 2: 
            winner = robot_2.name
        if max_bid_index == 3: 
            winner = robot_3.name
    print('{} won the bet and will make the color on {} points.'.format(winner, max_bid))
    if winner == player.name:
        player.colour_made = True
        game.color = input(f"What is the colour {player.name}?")
    if winner == robot_1.name:
        game.color = r1_mode_suit
    if winner == robot_2.name:
        game.color = r2_mode_suit
    if winner == robot_3.name:
        game.color = r3_mode_suit
    game.color = game.color.capitalize()

In [43]:
def flow_chart(r):
   
    global current_hand, game
    global df

    r.suits = list(r.cards["Suit"])
    if len(current_hand) == 0:
        if len(r.cards[r.cards['Points'] == 3]) > 0:
            card_to_play = r.cards[r.cards['Points'] == 3].sample(n = 1, random_state = None)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play)
    else: 
        current_suit = current_hand.iloc[0]['Suit']
        
        if current_suit in r.suits:
            how_many = r.suits.count(current_suit)

            if how_many == 1:
                card_to_play = r.cards[r.cards['Suit'] == current_suit]['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)

            if how_many > 1:

                if len(r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]) == 0:
                    cs = r.cards[r.cards['Suit'] == current_suit]
                    card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                    card_to_play = card_to_play.iloc[0]
                    print('{} plays {}'.format(r.name, card_to_play))
                    play_card(card_to_play, r)

                else: 
                    card_to_play = r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]['Name']
                    card_to_play = card_to_play.iloc[0]
                    print('{} plays {}'.format(r.name, card_to_play))
                    play_card(card_to_play, r)

        if not game.digged and current_suit not in r.suits:
            print(f'{r.name} has revealed the trump card. It is {game.color}')
            if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
                cs = r.cards[r.cards['Suit'] == game.color]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            else: 
                card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            game.digged = True
        elif game.digged and current_suit not in r.suits:
            if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
                cs = r.cards[r.cards['Suit'] == game.color]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            else: 
                card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)

In [37]:
current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [38]:
def play_game():
       #SETUP THE GAME
    setup()
    import warnings

    warnings.filterwarnings(
        "ignore",
        category=FutureWarning, 
        message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
    )

 
    #DO THE GLOBALS
    global player, robot_1, robot_2, robot_3, game, current_hand, last_hand, winner_index
    #print(player.cards)
    #PLAYE ALL THE 8 HANDS
    for i in range(0, 8):
        current_hand = pd.DataFrame(data = {}, columns = df.columns)
       
       
        if player.last_hand_won: 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":2,"robot_3":3}
            print("player")
        elif robot_1.last_hand_won: 
            
            flow_chart(robot_1)
            print(current_hand)
            flow_chart(robot_2)
            flow_chart(robot_3)
            print() 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            
            play_card(card, player)
            index_selctor = {"robot_1":0,"robot_2":1,"robot_3":2,"player":3}
            print("r1")
        elif robot_2.last_hand_won:
          
            flow_chart(robot_2)
            print(current_hand)
            flow_chart(robot_3)
            print() 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
          
            play_card(card, player)
            flow_chart(robot_1)
            index_selctor = {"robot_2":0,"robot_3":1,"player":2,"robot_1":3}
            
            print("r2")
        elif robot_3.last_hand_won:
            
            flow_chart(robot_3)
            print(current_hand)
            print() 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            index_selctor = {"robot_3":0,"player":1,"robot_1":2,"robot_2":3}
            print("r3")
        elif not robot_1.last_hand_won and not robot_2.last_hand_won and not robot_3.last_hand_won and not player.last_hand_won : 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
           
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":2,"robot_3":3}
            print("None")
        if game.digged and game.color in list(current_hand['Suit']):
            find_in = current_hand[current_hand['Suit'] == game.color]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        elif not game.color in list(current_hand['Suit']) or not game.digged:
            c_s = current_hand['Suit'].iloc[0]
            find_in = current_hand[current_hand['Suit'] == c_s]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        if winner_index == index_selctor["player"]: 
                for c in list(current_hand["Name"]): 
                    player.hands_won.append(c)
                player.last_hand_won = True
                robot_1.last_hand_won = False
                robot_2.last_hand_won = False
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_1"]: 
                for c in  list(current_hand["Name"]):
                    robot_1.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = True
                robot_2.last_hand_won = False
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_2"]: 
                for c in list(current_hand["Name"]):
                    robot_2.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = False
                robot_2.last_hand_won = True
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_3"]: 
                for c in list(current_hand["Name"]): 
                    robot_3.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = False
                robot_2.last_hand_won = False
                robot_3.last_hand_won = True
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        
        
        last_hand =  pd.DataFrame(data = {}, columns = df.columns)
        last_hand = current_hand
        current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [39]:
play_game()

What is your name? Aditya
Nice to meet you Aditya
Welcome to our newly made game of 29. We hope you enjoy it
              Name      Suit  Points
0   Jack of Spades    Spades    3.00
1  Queen of Hearts    Hearts    0.03
2   Jack of Hearts    Hearts    3.00
3   10 of Diamonds  Diamonds    1.00
How much do you want to bid?: 18
Aditya won the bet and will make the color on 18 points.
What is the colour Aditya?Hearts
Here are your cards Aditya: 
               Name      Suit  Points
0   Jack of Spades    Spades    3.00
1  Queen of Hearts    Hearts    0.03
2   Jack of Hearts    Hearts    3.00
3   10 of Diamonds  Diamonds    1.00
4   King of Hearts    Hearts    0.04
5    7 of Diamonds  Diamonds    0.01
6    Ace of Spades    Spades    1.10
7      7 of Spades    Spades    0.01
What card do you wanna play AdityaJack of Spades
Robot 1 plays Queen of Spades
Robot 2 plays 8 of Spades
Robot 3 has revealed the trump card. It is Hearts
Robot 3 plays 8 of Hearts
None
Robot 3 plays Jack of Diamonds
   

KeyboardInterrupt: Interrupted by user

In [ ]:
if game.digged and game.color in list(last_hand['Suit']):
            find_in = last_hand[last_hand['Suit'] == game.color]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
elif not game.color in list(last_hand['Suit']) or not game.digged:
            c_s = last_hand['Suit'].iloc[0]
            find_in = last_hand[last_hand['Suit'] == c_s]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index


In [ ]:
from turtle import Screen, Turtle

def handle_click(x, y):
    print(f"You clicked at ({x}, {y})")

screen = Screen()
screen.onclick(handle_click)  # Bind the function to screen clicks

screen.mainloop()


In [26]:
game.color

In [28]:
last_hand

,Name,Suit,Points
0,Jack of Diamonds,Diamonds,3.0
1,7 of Diamonds,Diamonds,0.01
2,King of Diamonds,Diamonds,0.04
3,7 of Hearts,Hearts,0.01


In [41]:
play_card("King of Spades",robot_1)

In [42]:
robot_1.cards

,Name,Suit,Points
8,9 of Spades,Spades,2.00
9,Queen of Clubs,Clubs,0.03
10,8 of Clubs,Clubs,0.02
13,10 of Spades,Spades,1.00
